<a href="https://colab.research.google.com/github/minyou2675/DL-study-179/blob/main/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive 
from google.colab import files

files.upload()


KeyboardInterrupt: ignored

In [3]:
%cp /content/movie_recommendation.zip /content/drive/MyDrive

In [4]:
%cd /content
!unzip '/content/drive/MyDrive/movie_recommendation.zip'

/content
Archive:  /content/drive/MyDrive/movie_recommendation.zip
  inflating: README.txt              
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [5]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/links.csv')
movies.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
links = pd.read_csv('/content/links.csv')
ratings = pd.read_csv('/content/ratings.csv')
tags = pd.read_csv('/content/tags.csv')
movies = pd.read_csv('/content/movies.csv')

In [7]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
df1 = pd.merge(ratings,movies,on='movieId',how='inner')
df1.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [11]:
df1 = df1.drop(['imdbId','tmdbId'],axis=1)
df1.head()

KeyError: ignored

In [12]:
df1.rename(columns={'userId':'user'},inplace=True)
df1.rename(columns={'movieId':'movie'},inplace=True)
df1.head()

,user,movie,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [13]:
X = df1.values
X[:,0]

array([1, 5, 7, ..., 610, 610, 610], dtype=object)

In [14]:
n = X.shape[0]
print(n)

100836


In [15]:
movie_names = movies.set_index('movieId')['title'].to_dict()
n_users = len(df1.user.unique())
n_items = len(df1.movie.unique())
print(n_users, n_items)

610 9724


In [23]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
  def __init__(self, n_users, n_items, n_factors =20):
    super().__init__()
    #create user embeddings
    self.user_factors = torch.nn.Embedding(n_users, n_factors)
    #create item embeddings
    self.item_factors = torch.nn.Embedding(n_items, n_factors)
    self.user_factors.weight.data.uniform_(0, 0.05)
    self.item_factors.weight.data.uniform_(0, 0.05)

  def forward(self, data):
    #matrix multiplication
    users, items = data[:,0], data[:,1]
    return (self.user_factors(users) * self.item_factors(items)).sum(1)

  def predict(self, user, item):
    return self.forward(user, item)

In [40]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

class Loader(Dataset):
  def __init__(self):
    self.ratings = ratings.copy()
    users = df1.user.unique()
    movies = df1.movie.unique()

    self.userid2idx = {o:i for i, o in enumerate(users)}
    self.movieid2idx = {o:i for i,o in enumerate(movies)}

    self.idx2userid = {i:o for o, i in self.userid2idx.items()}
    self.idx2movieid = {i:o for o, i in self.movieid2idx.items()}

    self.ratings.movieId = ratings.movieId.apply(lambda x: self.movieid2idx[x])
    self.ratings.userId = ratings.userId.apply(lambda x: self.userid2idx[x])

    self.x = self.ratings.drop(['rating','timestamp'], axis=1).values
    self.y = self.ratings['rating'].values
    self.x, self.y = torch.tensor(self.x), torch.tensor(self.y)

  def __getitem__(self, index):
    return (self.x[index], self.y[index])

  def __len__(self):
    return len(self.ratings)

In [42]:
num_epochs = 128
cuda = torch.cuda.is_available()

print('Is running on GPU:', cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
  if param.requires_grad:
      print(name, param.data)
  if cuda:
    model = model.cuda()


loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle = True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0162, 0.0172, 0.0349,  ..., 0.0382, 0.0039, 0.0225],
        [0.0063, 0.0210, 0.0437,  ..., 0.0363, 0.0496, 0.0366],
        [0.0035, 0.0158, 0.0067,  ..., 0.0418, 0.0291, 0.0421],
        ...,
        [0.0430, 0.0043, 0.0396,  ..., 0.0316, 0.0478, 0.0350],
        [0.0123, 0.0393, 0.0398,  ..., 0.0392, 0.0366, 0.0093],
        [0.0479, 0.0477, 0.0086,  ..., 0.0018, 0.0059, 0.0268]])
item_factors.weight tensor([[0.0294, 0.0025, 0.0325,  ..., 0.0249, 0.0198, 0.0172],
        [0.0382, 0.0200, 0.0387,  ..., 0.0221, 0.0472, 0.0060],
        [0.0471, 0.0088, 0.0082,  ..., 0.0226, 0.0358, 0.0114],
        ...,
        [0.0369, 0.0329, 0.0252,  ..., 0.0182, 0.0266, 0.0169],
        [0.0423, 0.0131, 0.0190,  ..., 0.0480, 0.0058, 0.0433],
        [0.0007, 0.0278, 0.0161,  ..., 0.0403, 0.0199, 0.0051]],
       device='cuda:0')


In [43]:
from tqdm import tqdm
for it in tqdm(range(num_epochs)):
  losses = []
  for x, y in train_loader:
    if cuda:
      x, y = x.cuda(), y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
  print("iter #{}", format(it), "LOSS:", sum(losses)/len(losses))


  1%|          | 1/128 [00:01<03:33,  1.68s/it]

iter #{} 0 LOSS: 11.065574382162335


  2%|▏         | 2/128 [00:02<03:04,  1.47s/it]

iter #{} 1 LOSS: 4.747645057397445


  2%|▏         | 3/128 [00:04<02:54,  1.40s/it]

...

iter #{} 125 LOSS: 0.32547452707354185


 99%|█████████▉| 127/128 [02:46<00:01,  1.30s/it]

iter #{} 126 LOSS: 0.32512247859297067


100%|██████████| 128/128 [02:48<00:00,  1.31s/it]

iter #{} 127 LOSS: 0.32452780728790964


In [44]:
c = 0
uw = 0
iw = 0
for name,param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)
    if c == 0:
      uw = param.data
      c += 1
    else:
        iw = param.data #weig

user_factors.weight tensor([[ 0.7396,  1.4572,  1.5752,  ...,  1.1006,  1.1056,  1.0895],
        [ 0.1522,  0.4590,  1.2808,  ...,  0.2429,  1.6283,  1.2521],
        [ 1.7419,  0.5475, -0.3486,  ...,  0.8494,  2.6998, -0.0406],
        ...,
        [ 0.8504,  0.6626,  0.9796,  ...,  2.1571,  1.1638,  0.9812],
        [ 1.1623,  0.2037,  3.4515,  ..., -0.1856,  1.2646, -0.4784],
        [ 0.7962,  2.4948,  1.7668,  ...,  0.8579,  0.2317,  1.1318]],
       device='cuda:0')
item_factors.weight tensor([[ 0.6318,  0.4255,  0.5805,  ...,  0.1209,  0.7861,  0.3773],
        [-0.1554,  0.6665,  0.4240,  ...,  0.5084,  0.2070,  0.4558],
        [ 0.5953,  0.3599,  0.1221,  ...,  0.6746,  0.7584,  0.5931],
        ...,
        [ 0.3628,  0.3578,  0.2998,  ...,  0.3442,  0.3515,  0.3416],
        [ 0.4235,  0.3946,  0.3738,  ...,  0.4294,  0.3880,  0.4253],
        [ 0.3660,  0.3913,  0.3863,  ...,  0.4034,  0.3837,  0.3685]],
       device='cuda:0')


In [45]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [46]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 10, random_state = 0).fit(trained_movie_embeddings)

In [48]:
for cluster in range(10):
  print(f"#Cluster {cluster}")
  movs = []
  for movidx in np.where(kmeans.labels_ ==cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings.loc[ratings['movieId'] == movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs,key = lambda tup: tup[1], reverse =True)[:10]:
    print("\t", mov[0])

#Cluster 0
	 Forrest Gump (1994)
	 Toy Story (1995)
	 Independence Day (a.k.a. ID4) (1996)
	 Apollo 13 (1995)
	 Fugitive, The (1993)
	 Aladdin (1992)
	 Lion King, The (1994)
	 Back to the Future (1985)
	 Dances with Wolves (1990)
	 Ace Ventura: Pet Detective (1994)
#Cluster 1
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Batman (1989)
	 Sixth Sense, The (1999)
	 True Lies (1994)
	 Speed (1994)
	 Gladiator (2000)
	 Men in Black (a.k.a. MIB) (1997)
	 Mission: Impossible (1996)
	 Beauty and the Beast (1991)
#Cluster 2
	 Shrek (2001)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 X-Men (2000)
	 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
	 Iron Man (2008)
	 Natural Born Killers (1994)
	 Shrek 2 (2004)
	 Mummy, The (1999)
	 Ice Age (2002)
	 Matrix Revolutions, The (2003)
#Cluster 3
	 Batman & Robin (1997)
	 Showgirls (1995)
	 Godzilla (1998)
	 Joe Dirt (2001)
	 Toys (1992)
	 Honey, I Blew Up the Kid (1992)
